# Sentiment Classification Using BERT

We will use IMDB Movie Reviews Dataset

In [1]:
!pip install ktrain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.3/25.3 MB 15.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 14.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 kB 48.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for ktrain: filename=ktrain-0.41.3-py3-none-any.whl size=25316960 sha256=9d0bf317e40a1f1fb5a3096553bf20d8a7dec421b1bb34f18221ec3d27e79de3
  Stored in directory: /root/.cache/pip/wheels/fc/76/11/5b953090eebf531f660948a30cd26e70260619f6480f186a5a
  Created wheel for keras_bert: filename=

In [2]:
import tensorflow as tf

In [3]:
tf.__version__

'2.15.0'

In [4]:
!git clone https://github.com/tharani001/IMDB-Dataset.git

Cloning into 'IMDB-Dataset'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 4 (delta 0), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (4/4), 25.78 MiB | 25.53 MiB/s, done.


In [5]:
import pandas as pd
import numpy as np
import ktrain
from ktrain import text

In [6]:
data_train = pd.read_excel('/content/IMDB-Dataset/train.xlsx', dtype = str)

In [7]:
data_test = pd.read_excel('/content/IMDB-Dataset/test.xlsx', dtype = str)

In [8]:
data_train.tail()

,Reviews,Sentiment
24995,Everyone plays their part pretty well in this ...,pos
24996,It happened with Assault on Prescient 13 in 20...,neg
24997,My God. This movie was awful. I can't complain...,neg
24998,"When I first popped in Happy Birthday to Me, I...",neg
24999,"So why does this show suck? Unfortunately, tha...",neg


In [9]:
data_test.head()

,Reviews,Sentiment
0,Who would have thought that a movie about a ma...,pos
1,After realizing what is going on around us ......,pos
2,I grew up watching the original Disney Cindere...,neg
3,David Mamet wrote the screenplay and made his ...,pos
4,"Admittedly, I didn't have high expectations of...",neg


In [10]:
data_test.shape, data_train.shape

((25000, 2), (25000, 2))

# `Data Preprocessing and Train-Test Split`

In [12]:
(X_train, y_train), (X_test, y_test), preproc = text.texts_from_df(train_df=data_train,
                                                                   text_column = 'Reviews',
                                                                   label_columns = 'Sentiment',
                                                                   val_df = data_test,
                                                                   maxlen = 500,
                                                                   preprocess_mode = 'bert')

['neg', 'pos']
   neg  pos
0  1.0  0.0
1  1.0  0.0
2  1.0  0.0
3  1.0  0.0
4  1.0  0.0
['neg', 'pos']
   neg  pos
0  0.0  1.0
1  0.0  1.0
2  1.0  0.0
3  0.0  1.0
4  1.0  0.0
preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In the `text_from_df()` the argument called `maxlen` specifies the number of words or the word limit for each review in the dataset to predict the corresponding label.

# Model Training

`text_classifier` just compiles the model

compiling a model means setting up the model's parameters and hyperparameters before training it on a dataset. This step is crucial because it determines how the model will learn from the data and how it will make predictions

In [13]:
model = text.text_classifier(name = 'bert',
                             train_data = (X_train, y_train),
                             preproc = preproc)

Is Multi-Label? False
maxlen is 500


/usr/local/lib/python3.10/dist-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer GlorotNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


done.


`get_learner` creates an instance to finetune the pretrained model

In [14]:
learner = ktrain.get_learner(model=model, train_data=(X_train, y_train),
                   val_data = (X_test, y_test),
                   batch_size = 6)

After training the bert model on train data, we get learner instance to finetune the bert model.

# Best Learning Rate

find out best learning rate?
learner.lr_find()
learner.lr_plot()

it may take days or many days to find out when doing it for the first time.

## Based on Research, lr=2e-5 works better for sentiment classification

In [ ]:
learner.fit_onecycle(lr = 2e-5, epochs = 1)



begin training using onecycle policy with max lr of 2e-05...
Train on 25000 samples, validate on 25000 samples
25000/25000 [==============================] - 3500s 140ms/sample - loss: 0.1361 - accuracy: 0.9517 - val_loss: 0.0487 - val_accuracy: 0.9872


In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc)

In [ ]:
data = ['this movie was horrible, the plot was really boring. acting was okay',
        'the fild is really sucked. there is not plot and acting was bad',
        'what a beautiful movie. great plot. acting was good. will see it again']

In [ ]:
predictor.predict(data)

['neg', 'neg', 'pos']

In [ ]:
predictor.predict(data, return_proba=True)

array([[0.99797565, 0.00202436],
       [0.99606663, 0.00393336],
       [0.00292433, 0.9970757 ]], dtype=float32)

In [ ]:
predictor.get_classes()

['neg', 'pos']

In [ ]:
predictor.save('/content/bert')

In [ ]:
!zip -r /content/bert.zip /content/bert

  adding: content/bert/ (stored 0%)
  adding: content/bert/tf_model.h5 (deflated 11%)
  adding: content/bert/tf_model.preproc (deflated 52%)


In [ ]:
predictor_load = ktrain.load_predictor('/content/bert')

In [ ]:
predictor_load.get_classes()

['neg', 'pos']

In [ ]:
predictor_load.predict(data)

['neg', 'neg', 'pos']